In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 sentencepiece==0.1.99

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:

DataSet = load_dataset("messawey/EG_Historical_QA")

train_data, test_data = train_test_split(DataSet['train'], test_size=0.2, random_state=2024)

Generating train split:   0%|          | 0/420 [00:00<?, ? examples/s]

In [8]:
train_data['MergedColumn'][0]

'<s>[INST] ما هي أهمية كنوز توت عنخ امون؟ [/INST] ترجع أهمية مجموعة الملك توت عنخ آمون إلى العديد من الأسباب؛ أولها أن كنز الملك توت عنخ أمون هو أكمل كنز ملكي عُثر عليه ولا نظير له، إذ يتكون من ثلاثمائة وثمان وخمسين قطعة تشمل القناع الذهبي الرائع وثلاثة توابيت على هيئة الإنسان، أحدها من الذهب الخالص والآخران من خشب مذهب. ثانياً: أن تلك الأمتعة ترجع إلى الأسرة الثامنة عشرة أشهر وأزهى عصور الدولة الحديثة حيث انفتحت البلاد على أقاليم الشرق الأدنى القديم بفضل الحملات العسكرية والعلاقات التجارية من تصدير واستيراد للموارد والمنتجات المصنعة ونشاط أهل الحرف والفنانين. وأخيراً: أن هذه المجموعة الهائلة قد ظلت في مصر، وتوضح كيف كان القبر الملكي يجهز'

In [10]:
model_name = "hpcai-tech/Colossal-LLaMA-2-7b-base"

# Fine-tuned model name
new_model = "llama_2_EG_Historical_QA"

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# TrainingArguments parameters

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 10

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 75

# Log every X updates steps
logging_steps = 25

# Maximum sequence length to use
max_seq_length = 210

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [11]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/600 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/21.3k [00:00<?, ?B/s]

pytorch_model-00001.bin:   0%|          | 0.00/1.07G [00:00<?, ?B/s]

pytorch_model-00002.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/745 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


## Per-Training Results

In [12]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt ="متى حكمت حتشبسوت مصر؟"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s>[INST] متى حكمت حتشبسوت مصر؟ [/INST] 视频提及：埃及是一个位于非洲东北部的国家，它西面与利比亚接壤，西南与苏丹交界，北临地中海，东面是红海，与巴勒斯坦、以色列、约旦、沙特阿拉伯隔海相望。 nobody：埃及是一个位于非洲东北部的国家，它西面与利比亚接壤，西南与苏丹交界，北临地中海，东面是红海，与巴勒斯坦、以色列、约旦、沙特阿拉伯隔海相望。


In [13]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "ما هو الغرض من بناء هرم خوفو؟ "
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] ما هو الغرض من بناء هرم خوفو؟  [/INST]视频提及：我们的建筑是由砖块、混凝土、钢筋、玻璃、木材、石头、金属等等材料组成的，这些材料是如何被加工成各种各样的建筑形态的呢？


## Training

In [14]:
text_lengths = []

# Iterate over the dataset and calculate the length of each text
for example in DataSet['train']:
    text = example['MergedColumn']
    text_length = len(text.split())
    text_lengths.append(text_length)

# Calculate the maximum length
max_length = max(text_lengths)

print(f"The maximum text length is: {max_length} words")

The maximum text length is: 212 words


In [15]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
)

In [16]:
from sklearn.metrics import accuracy_score
def custom_evaluation_metric(predictions, labels):
    accuracy = accuracy_score(predictions, labels)

In [17]:
# Set supervised fine-tuning parameter
trainer = SFTTrainer(
    model=model,
    train_dataset=DataSet['train'],  # Access the 'train' key
    peft_config=peft_config,
    dataset_text_field="MergedColumn",  # Use the correct column name
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
    compute_metrics=custom_evaluation_metric,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/420 [00:00<?, ? examples/s]

In [18]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=DataSet['train'],  # Access the 'train' key
    peft_config=peft_config,
    dataset_text_field="MergedColumn",  # Use the correct column name
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

In [19]:
# Train model
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.7117, 'learning_rate': 0.0002, 'epoch': 0.24}
{'loss': 1.3697, 'learning_rate': 0.0002, 'epoch': 0.48}
{'loss': 1.11, 'learning_rate': 0.0002, 'epoch': 0.71}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.0465, 'learning_rate': 0.0002, 'epoch': 0.95}
{'loss': 0.9937, 'learning_rate': 0.0002, 'epoch': 1.19}
{'loss': 0.9974, 'learning_rate': 0.0002, 'epoch': 1.43}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.2553, 'learning_rate': 0.0002, 'epoch': 8.81}
{'loss': 0.2548, 'learning_rate': 0.0002, 'epoch': 9.05}
{'loss': 0.1923, 'learning_rate': 0.0002, 'epoch': 9.29}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.2084, 'learning_rate': 0.0002, 'epoch': 9.52}
{'loss': 0.2198, 'learning_rate': 0.0002, 'epoch': 9.76}
{'loss': 0.2202, 'learning_rate': 0.0002, 'epoch': 10.0}
{'train_runtime': 2814.865, 'train_samples_per_second': 1.492, 'train_steps_per_second': 0.373, 'train_loss': 0.5895237927209763, 'epoch': 10.0}


TrainOutput(global_step=1050, training_loss=0.5895237927209763, metrics={'train_runtime': 2814.865, 'train_samples_per_second': 1.492, 'train_steps_per_second': 0.373, 'train_loss': 0.5895237927209763, 'epoch': 10.0})

In [20]:
trainer.model.save_pretrained(new_model)

## Results

In [21]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt ="متى حكمت حتشبسوت مصر؟"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s>[INST] متى حكمت حتشبسوت مصر؟ [/INST] حكمت حتشبسوت مصر حوالي عام 1479-1458 قبل الميلاد.
توليت حكم لابنه نيرون.
اعترف بابا الأحد بأسمه حتشبسوت.
اعترف بابا الآخر بأسمه رعامة.
اعترف بابا الثالث بأسمه نيرون.
اعترف بابا الر


In [22]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "أين دفنت حتشبسوت؟"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] أين دفنت حتشبسوت؟ [/INST] في وادي الملوك في الحضارة المصرية القديمة.
انتشرت الحضارة المصرية من أجل وادي ذات عدد من الملوك وأسرات مختلفة، وتم نقل الحضارة إلى أوايت أخرى من الإمبراطورية.
استمرت الحضارة عدة أوروبا من الأطول


In [23]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "ما هو الغرض من بناء هرم خوفو؟ "
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] ما هو الغرض من بناء هرم خوفو؟  [/INST] كان الغرض الرئيس من بناء الهرم هو استخدامه للتعايش الحي، بما في ذلك التراث الديني والتعايش الحي.
التراث الديني كان من أرباح الأناث والتعايش الحي وتأثير التحول الثقافي.
التعايش الحي كان من أرب
